# Eight Ever, Nine Never

### Setting Things Up for Python ...

This notebook uses _Python_ so we first set up a few things for that environment (note that this section of the notebook is tagged to not appear when we export the document to HTML or other formats):

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import scipy.special
import scipy.stats as stats

%matplotlib inline

We also define a little function that pretty prints large numbers using commas to make them more readable so e.g. 8753912376 becomes 8,753,912,376 and small numbers as percentages ao 0.723 becomes 72.3%. 

In [13]:
def comma_print(x):   print(f'{x:,}')
def percent_print(x): print(f'{x:.1%}') 
def pretty_print(x):  percent_print(x) if 0 <= x <= 1 else comma_print(x)

## Introduction

Bridge is a game played by 4 players using a normal deck of 52 cards. The full deck is used so each player is dealt 13 cards. 

How many distinct Bridge _hands_ are there? Each hand has 13 cards picked from a shuffled deck of 52 candidates. The specific order in which the cards are dealt is irrelevant — after all, typically the first thing you do when you pick up the hand is to immediately sort the cards into suits so clearly it isn't important if you pick up the 10$\clubsuit$ first followed later by the Q$\heartsuit$ or vice versa.  Mathematically speaking we are only interested in the number of _combinations_ of 13 cards one can make from a pack of 52 cards (as opposed to the number of _permutations_ of 13 cards where the order of selection was important for some reason).

We will use $C(n,p)$ to denote the number of combinations of $p$ objects one can make from  $n$ candidates. Combinations are well understood in mathematics and $C(n,p)$ has the formula:
$$
  C(n,p) = \frac{n!}{p! \, (n-p)!},
$$
where $n!$ stands for _factorial_ $n$ defined by
$$
  n! = n \times (n-1) \times (n-2) \times \dots \times 2 \times 1.
$$

There is a handy _Python_ function we can use to compute $C(n,p)$:

In [14]:
def C(n,p): return scipy.special.comb(n,p,exact=True)

So the number of distinct Bridge hands, $N_h$ is

In [15]:
number_of_hands = C(52, 13)
pretty_print(number_of_hands)

635,013,559,600


or roughly 635 billion.

However, that's only part of the story as the four players at the Bridge table each has their own 13-card hand and what really matters for any game is the set of the four hands. The Bridge term used for that set is a _board_ and there are a truly vast number of possible boards. 

To see that you might think of the first player getting her 13 cards from the full 52 card deck, the second getting 13 cards from the remaining 39 cards, the third getting 13 from the remaining 26 cards, and the final 13 cards going to the last player. This means that the number of possible boards is

$$
  N_b = C(52,13) \times C(39,13) \times C(26,13) \times C(13,13),
$$

which we can compute using our $C$ function:

In [16]:
number_of_boards = C(52,13) * C(39,13) * C(26,13) * C(13,13)
pretty_print(number_of_boards)

53,644,737,765,488,792,839,237,440,000


That is, there are $5.36 \times 10^{28}$ possible boards which is a staggeringly large number. There is no worry that all possible boards will be analyzed completely any time soon — Bridge will continue to be a game requiring on-the-spot analysis and skill!

## A Suit Break Example

To motivate things we will look at an example where we are playing in no-trump and planning our line of play at the start of the game just after the dummy has put down his hand. 

Suppose as declarer we hold the $\spadesuit$AKQJ5 in our own hand and sees the $\spadesuit$432 in the dummy's hand on the table. The opponents hold the remaining five spades (the $\spadesuit$T9876 in this example) between them. We will try to make all five spade tricks by leading out the top spades one after another in an effort to exhaust the opponent's holding in the suit. Will that strategy work? 

It is easy to see that the declarer is guaranteed to make all the spade tricks as long as the missing five cards are **not** distributed 5-0 or 0-5. That 5-0 _break_ is defined to mean that the declarer's left hand opponent (LHO) has all five spades and the right hand opponent (RHO) has none. The 0-5 break equivalently means that the LHO has no spades and the RHO has all five. With a 5-0 or 0-5 break the declarer can only make four spade tricks as the opponents can block her fifth trick by hanging on to their 10$\spadesuit$.

What are the odds of getting that bad 5-0 or 0-5 break?

We are at the start of play so the opponents hold a total of 26 cards between them. Repeating our earlier argument for four hands we might think of one opponent getting their 13 cards from the 26 and the remaining 13 going to her partner. So with no constraints, the number of distinct hands the opposition can have is

$$
    C(26,13) \times C(13,13)
$$ 
or

In [52]:
pretty_print(C(26,13)*C(13,13))

10,400,600


which at a mere 10 million odd seems quite manageable after those giant numbers we encountered earlier!

In the case we are considering, the opponents hold 5 spades and 21 others. What is the probability that the LHO has exactly 5 spades and by extension 8 non-spade cards (in which case of course the RHO will have exactly 0 spades and 13 others). 

Well the LHO gets a  selection of 5 spades from the 5 outstanding. The LHO also gets 8 other cards from the 21 non-spades outstanding. The total number of distinct hands of this form possible for the LHO is:

$$
C(5,5) \times C(21,8)
$$
or

In [53]:
pretty_print(C(5,5)*C(21,8))

203,490


Hence the *probability* that the LHO has all five spades (and by extension that the RHO has none) is 

$$
\frac{C(5,5) \times C(21,8)}{C(26,13)}
$$
which is just

In [54]:
pretty_print(C(5,5)*C(21,8) / C(26,13))

2.0%


So the chances of getting a 5-0 break is 2%. Repeating the argument you see that the chance of a 0-5 break is also 2%. Therefore the probability of getting any bad break is 4%. This means that 96% of the time the declarer can be confident of taking all five spade tricks so will feel very unlucky if things go badly in that suit!

## Another Suit Break Example

For our second example consider a no-trump game where declarer holds $\spadesuit$AKQ65 in her own hand and has the $\spadesuit$432 in the dummy's hand on the table. Her opponents have the remaining five spades (the $\spadesuit$JT987 in this example) between them. Even missing the $\spadesuit$J, declarer still has a very powerful spade suit and will hope to make all five spade tricks by leading out the top cards one after another in an effort to exhaust her opponent's holding in the suit. How will that strategy work this time? 

It is easy to see that she will make all five tricks as long as the spades break either 3-2 or 2-3 as in those cases the three tricks taken by the $\spadesuit$AKQ will leave the opponents with no more spades and our declarer can safely cash in her two little ones. 

By design in this example the opponents hold all the intermediate spades so if the suit breaks 5-0 or 0-5 then the declarer can only ever make three tricks in spades. If the suit breaks 4-1 or 1-4 then the declarer can at most make four tricks in spades.

What are the chances of getting the good break 3-2?

Once again the opponents have 26 cards between them and 5 of those are spades and 21 are not. In this break the LHO will have 3 of 5 outstanding spades and 10 other cards. The number of distinct hands of that form for the LHO is then: 

$$
C(5,3) \times C(21,10)
$$
or

In [55]:
pretty_print(C(5,3)*C(21,10))

3,527,160


Hence the probability that the LHO has three spades (and by extension that the RHO has two) is 

$$
\frac{C(5,3) \times C(21,10)}{C(26,13)}
$$
which is just

In [56]:
pretty_print(C(5,3)*C(21,10) / C(26,13))

33.9%


So the chances of getting a 3-2 break is 33.9%. Repeating the argument you see that the chance of a 2-3 break is also 33.9%. This means that the combined odds of getting that _good_ 2-3 or 3-2 break in spades is 67.8%. Not bad at all, but it does means that the there is roughly one chance in three that the declarer does _not_ get all five spade tricks so might want to have a backup plan in her pocket just in case!

## Suit Break Probabilities at the Start of Play

Calculating the probabilities of how the missing cards in a suit can break across the hands of two opponents is a fundamental problem for the mathematics of Bridge. Following our examples above we can readily create a function to compute the probability of any break across two hands at the start of play. The function will take two arguments, for example, if those are 4 and 3 then the function returns the probability of 4-3 break for the case where the opponents hold a total of 7 cards in the suit between them. 

In [57]:
def combinations_with_break(m,n): return C(m+n,m)*C(26-m-n, 13-m)
def probability_of_break(m,n):    return combinations_with_break(m,n) / C(26,13)

As a quick check we can re-compute an earlier example

In [58]:
pretty_print(probability_of_break(2,3))

33.9%


We can use our `combinations_with_break(m,n)` and `probability_of_break(m,n)` functions to create a table of all the possible breaks for any given number of missing cards:

In [59]:
def table_of_breaks_missing(m):
    
    # Iterate over all the possible breaks for the missing m cards from 0–m to m–0.
    # Use our `probabilityOfBreak` function to get the associated probability for each one
    data = [ ("{}–{}".format(k,m-k),
              combinations_with_break(k,m-k), 
              probability_of_break(k,m-k)) 
            for k in range(m+1) ]
    
    # Set up the table for that data with some labeled columns
    df = pd.DataFrame(data, columns = ["Break", "Combinations", "Probability"])
    
    # Format that table in a nice way
    table_properties = { "width": "20em", "text-align": "center" }
    header_style     = { "selector": "th", "props": [("text-align", "center")] }
    caption_style    = { "selector": "caption", 
                         "props": [("font-weight", "bold"), 
                                   ("font-style",  "italic"), 
                                   ("font-size",   "110%"),
                                   ("text-align",  "center")] }

    styled_DF = df.style.hide_index().\
                  set_caption("Break Table for {} Missing Cards".format(m)).\
                  format({"Combinations": "{:,}"}).\
                  format({"Probability": "{:.2%}"}).\
                  set_properties(**table_properties).\
                  set_table_styles([header_style, caption_style])
    
    return styled_DF

For example, if we are missing just 1 card the table looks like:

In [60]:
table_of_breaks_missing(1)

Break,Combinations,Probability
0–1,"5,200,300",50.00%
1–0,"5,200,300",50.00%


which is what you would expect—one of the two opponents has the missing card and in the absence of any other information it is a straight coin-flip to guess which one. 

In contrast if we look at the case where we are missing 2 cards

In [61]:
table_of_breaks_missing(2)

Break,Combinations,Probability
0–2,"2,496,144",24.00%
1–1,"5,408,312",52.00%
2–0,"2,496,144",24.00%


which is perhaps just a little off from what your initial instinct might lead you to think. For if we label the missing cards by  $a$ and $b$ then you might reason there are only four scenarios. Namely, the LHO has both cards, the LHO has just one of them (that's two more possibilities), or the LHO has neither card. With no other information these seem to be equally likely and you might conclude that the probability of a 2-0 or 0-2 break is 25% and the probability of a 1-1 break is 50%. That's _close_ to what we have in our table but obviously not an exact match. What gives?

The thing missing in the simple analysis is the identities of all the _other_ cards in the two hands. So when the LHO has both $a$ and $b$ she also has 11 other cards from a pool of 24 (i.e. the 26 cards the opponents have between them less the $a$ and $b$ cards). Instead of just _one_ scenario where the LHO has both cards there are in fact

$$ 
    C(24,11) = 2,496,144
$$

such 2-0 break scenarios. It is easy to see that there are exactly the same number of 0-2 break scenarios. So the total number of scenarios that yield a 2-0 or 0-2 break is 4,992,288

Similarly instead of there being just _two_ scenarios where the LHO has just one of the two cards you should think that if she has one of $a$ or $b$ then she also has 12 other cards from that pool of 24 so there are 

$$
    2 C(24,12) = 5,408,312
$$

scenarios where we get a 1-1 break. That is a bit more than the 4,992,288 scenarios where we _don't_ get a 1-1 break! This means that at the start of play the probability of getting a 1-1 break is in fact slightly higher than 50%!

Moving on to the situation where we are missing 3 cards we have:

In [49]:
table_of_breaks_missing(3)

Break,Combinations,Probability
0–3,"1,144,066",11.00%
1–2,"4,056,234",39.00%
2–1,"4,056,234",39.00%
3–0,"1,144,066",11.00%


and as we mentioned earlier there is an excellent 78% chance the declarer will see one of the two most even breaks (namely 1-2 or 2-1).

For 4 missing cards we have:

In [62]:
table_of_breaks_missing(4)

Break,Combinations,Probability
0–4,"497,420",4.78%
1–3,"2,586,584",24.87%
2–2,"4,232,592",40.70%
3–1,"2,586,584",24.87%
4–0,"497,420",4.78%


so the chance of a perfect 2-2 split is only about 41%.

How about 5 missing cards?

In [63]:
table_of_breaks_missing(5)

Break,Combinations,Probability
0–5,"203,490",1.96%
1–4,"1,469,650",14.13%
2–3,"3,527,160",33.91%
3–2,"3,527,160",33.91%
4–1,"1,469,650",14.13%
5–0,"203,490",1.96%


We see that the most even possible breaks (either 2-3 or 3-2) capture roughly 68% of the scenarios. That is very useful information for the declarer. 

## "Eight Ever, Nine Never"

A declarer will often have a total of 8 or 9 cards in some key suit between her own and dummy's hand and squeezing the maximum number of tricks out of this suit is often critical for success. If the holding is headed by the Ace and King but is missing the Queen she is faced with a classic choice of playing out the Ace and King and hoping that the Queen will _drop_, or instead trying to _finesse_ the Queen to prevent the opponents capturing a trick. Of course there may be other information the declarer can glean from the bidding or the play of the other suits to inform this decision, but every player at some point in their Bridge career learns that absent that, the rule to follow is _eight ever, nine never_, meaning that you should always go for the finesse if you hold just 8 cards in the suit and always go for the drop if you are holding 9 cards.  Where does that wisdom that come from and just how good is it anyway?

### Holding 9 — The Drop

Consider the case where the declarer has 9 trump with say AKJT9 opposite the 5432 in dummy. The simplest strategy is to play for the drop by cashing in the Ace and King hoping the Queen falls. This will happen if the Queen is a singleton or one of a doubleton. What are the odds of that being the case?
 
The opponents hold 4 trumps between them so the Queen will appear in a doubleton is if we get an even 2-2 break. That has the probability

In [31]:
pretty_print(probability_of_break(2,2))

40.7%


The Queen will also fall if it is a singleton which will be the case in one quarter of all the 3-1 breaks and one quarter all the 1-3 breaks which is

In [32]:
pretty_print(0.25*(probability_of_break(3,1) + probability_of_break(1,3)))

12.4%


So all in all the drop will work some 53.1% of the time. Not exactly a bet-the-house percentage but the perhaps better odds than you get from trying the finesse?

### Holding 9 — The Finesse

As before the declarer has 9 cards in the suit; the AKJ76 opposite the 5432 in dummy and we'll assume that she has as many entries to dummy as needed. The basic finesse strategy is to first cash in the Ace and then cross to the dummy and play low towards the Jack (obviously covering the Queen if the RHO plays it), return to dummy and repeat. The strategy succeeds if the opponents cards in the suit are divided in any of the following ways: 

|  Scenario  | LHO | RHO | Comment                                                           |
|:--:|:---:|:---:| ------------------------------------------------------------------------- |
| 1. | Q   | T98 | The Q falls immediately to the A. The T9 are then mopped up by the KJ     |
| 2. | T98 | Q   | The same thing.                                                           |
| 3. | xx  | Qx  | x stands for any of the T98. The Q falls on the second play from dummy.   |
| 4. | x   | Qxx | The Q falls on the third round! The _finesse_ succeeds, the _drop_ fails. |

On the other hand the finesse fails if:

| Scenario  | LHO  | RHO  | Comment                                                            |
|:--:|:----:|:----:| ------------------------------------------------------------------------- |
| 5. | Qx   | xx   | The _drop_ strategy works in this situation but the _finesse_ fails!      |
| 6. | Qxx  | x    | Declarer cannot capture the Q in this situation. Both strategies fail.    |
| 7. | QT98 | -    | Same thing.                                                               |
| 8. | -    | QT98 | Good try but the dummy is too short to finesse our way to success.        |

Concentrating on the first table where the finesse succeeds we see that 1. is a subset of a 1-3 break where the Queen is the singleton (so 25% of all 1-3 breaks). Similarly 2. happens in 25% of all 3-1 breaks. There are 6 possible 2-2 breaks and scenario 3. represents three of them as the x in Qx can be one of T98. So scenario 3. occurs in 50% of all 2-2 breaks. Finally scenario 4. is a another 1-3 break and three of the four 1-3 breaks will have this form. 

So the probability of success for the finesse is:

In [33]:
pretty_print(0.25*probability_of_break(1,3) + 
             0.25*probability_of_break(3,1) + 
             0.50*probability_of_break(2,2) + 
             0.75*probability_of_break(1,3))

51.4%


i.e. the _drop_ has a 53.1% chance of success whereas the _finesse_ has a 51.4% chance. There is then just a slight preference for playing for the drop when you hold nine cards in the suit and need to capture the missing Queen. However, both strategies are really not much better than a coin flip! Ideally you might try and garner some other information to inform your play before embarking on either line of attack. 

The place where this type of decision has to be made early is often when you are drawing trump. The declarer is usually worried about the defense getting a ruffing trick and cannot wait before playing out the trump suit. In that case the optimal strategy (by a whisker) is to play for the drop. 

Looking at our tables above we see that the defense will always get a trump trick in scenarios 6, 7, and 8. The _finesse_ works in scenario 4 where the _drop_ fails while the exact opposite happens in scenario 5. For the particular layout here where we have 9 cards in all it turns out that scenario 5 is slightly more frequent than scenario 4 and hence the _drop_ is favored.

## Breaks for Any Number of Missing Cards at the Start of Play

In [35]:
for missing in range(1,14): display(table_of_breaks_missing(missing))
    

Break,Probability
0–1,50.00%
1–0,50.00%


Break,Probability
0–2,24.00%
1–1,52.00%
2–0,24.00%


Break,Probability
0–3,11.00%
1–2,39.00%
2–1,39.00%
3–0,11.00%


Break,Probability
0–4,4.78%
1–3,24.87%
2–2,40.70%
3–1,24.87%
4–0,4.78%


Break,Probability
0–5,1.96%
1–4,14.13%
2–3,33.91%
3–2,33.91%
4–1,14.13%
5–0,1.96%


...

Break,Probability
0–9,0.02%
1–8,0.54%
2–7,4.28%
3–6,15.71%
4–5,29.45%
5–4,29.45%
6–3,15.71%
7–2,4.28%
8–1,0.54%
9–0,0.02%


Break,Probability
0–10,0.01%
1–9,0.17%
2–8,1.89%
3–7,9.24%
4–6,23.10%
5–5,31.18%
6–4,23.10%
7–3,9.24%
8–2,1.89%
9–1,0.17%


Break,Probability
0–11,0.00%
1–10,0.05%
2–9,0.72%
3–8,4.76%
4–7,15.88%
5–6,28.58%
6–5,28.58%
7–4,15.88%
8–3,4.76%
9–2,0.72%


Break,Probability
0–12,0.00%
1–11,0.01%
2–10,0.23%
3–9,2.12%
4–8,9.53%
5–7,22.87%
6–6,30.49%
7–5,22.87%
8–4,9.53%
9–3,2.12%


Break,Probability
0–13,0.00%
1–12,0.00%
2–11,0.06%
3–10,0.79%
4–9,4.92%
5–8,15.93%
6–7,28.31%
7–6,28.31%
8–5,15.93%
9–4,4.92%
